In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from shapely.geometry import Point
from geopy.distance import geodesic
from sklearn.cluster import KMeans

# Load booth data
assam_booths = gpd.read_file('/Users/amartyaanand/Desktop/assam booth mapping/assam/assam.booth.shp')

# Load AC boundaries
ac_boundaries = gpd.read_file('/Users/amartyaanand/Desktop/assam booth mapping/assam/assam.assembly.shp')

# Ensure same CRS
assam_booths = assam_booths.to_crs(ac_boundaries.crs)

# Spatial join to keep booths within AC boundaries
booths_with_ac = gpd.sjoin(assam_booths, ac_boundaries, how='inner', predicate='within')

# Use correct column names for filtering
ac_column_booths = 'ac_left'  # from assam_booths after spatial join
ac_column_boundaries = 'ac'   # from ac_boundaries

unique_acs = booths_with_ac[ac_column_booths].unique()

# Define colors for clusters
colors = [
    'red', 'blue', 'green', 'purple', 'orange',
    'darkred', 'lightblue', 'lightgreen', 'pink', 'gray'
]

for ac_name in unique_acs:
    ac_booths = booths_with_ac[booths_with_ac[ac_column_booths] == ac_name]
    ac_boundary = ac_boundaries[ac_boundaries[ac_column_boundaries] == ac_name]

    if ac_booths.empty or ac_boundary.empty:
        continue

    # Filter booths to keep only those within the AC boundary polygon
    ac_polygon = ac_boundary.geometry.unary_union
    ac_booths = ac_booths[ac_booths.geometry.within(ac_polygon)]

    if ac_booths.empty:
        continue

    # Create map centered on AC booths
    ac_booths_proj = ac_booths.to_crs(epsg=3857)
    center = ac_booths_proj.geometry.centroid.unary_union.centroid
    center_lonlat = gpd.GeoSeries([center], crs="EPSG:3857").to_crs(epsg=4326).geometry[0]
    m = folium.Map(location=[center_lonlat.y, center_lonlat.x], zoom_start=12)

    # Add AC boundary polygon
    folium.GeoJson(
        ac_boundary.geometry,
        name=f"{ac_name} Boundary",
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 2,
            'dashArray': '5, 5'
        }
    ).add_to(m)

    # Extract coordinates for clustering
    coords = ac_booths.geometry.apply(lambda geom: (geom.centroid.x, geom.centroid.y) if geom.geom_type != 'Point' else (geom.x, geom.y))
    X = np.array(coords.tolist())

    # KMeans clustering
    kmeans = KMeans(n_clusters=10, random_state=42)
    ac_booths['cluster'] = kmeans.fit_predict(X)

    # Plot booths as circles colored by cluster
    for _, row in ac_booths.iterrows():
        point = row.geometry if row.geometry.geom_type == 'Point' else row.geometry.centroid
        cluster_color = colors[row['cluster'] % len(colors)]
        folium.CircleMarker(
            location=[point.y, point.x],
            radius=2,
            color=cluster_color,
            fill=True,
            fill_color=cluster_color,
            fill_opacity=0.8,
            tooltip=row[ac_column_booths]
        ).add_to(m)

    selected_pairs = []

    # Select pairs per cluster with distance between 500m and 2km and near cluster centroid
    for cluster_id in range(10):
        cluster_booths = ac_booths[ac_booths['cluster'] == cluster_id]
        if cluster_booths.empty:
            continue

        # Calculate cluster centroid
        cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
        cluster_centroid_latlon = gpd.GeoSeries([cluster_centroid], crs=ac_booths.crs).to_crs(epsg=4326).geometry[0]

        coords_cluster = cluster_booths.geometry.apply(lambda geom: (geom.centroid.y, geom.centroid.x) if geom.geom_type != 'Point' else (geom.y, geom.x)).tolist()

        # Calculate distance of each point to cluster centroid
        distances_to_centroid = [geodesic((lat, lon), (cluster_centroid_latlon.y, cluster_centroid_latlon.x)).meters for lat, lon in coords_cluster]

        # Sort points by distance to centroid
        sorted_indices = np.argsort(distances_to_centroid)

        found = False
        # Try pairs starting from points closest to centroid
        for i in sorted_indices:
            for j in sorted_indices:
                if i >= j:
                    continue
                dist = geodesic(coords_cluster[i], coords_cluster[j]).meters
                # Check distance between points and proximity to centroid
                if 500 <= dist <= 2000:
                    # Also check if both points are within 1000m of centroid (near centroid)
                    if distances_to_centroid[i] <= 1000 and distances_to_centroid[j] <= 1000:
                        selected_pairs.append((coords_cluster[i], coords_cluster[j], cluster_id, dist))
                        found = True
                        break
            if found:
                break

    # Plot selected pairs with pin points and polylines
    for booth1, booth2, cluster_id, dist in selected_pairs:
        folium.Marker(location=booth1, icon=folium.Icon(color='blue', icon='info-sign'), popup=f'Cluster {cluster_id} Booth 1').add_to(m)
        folium.Marker(location=booth2, icon=folium.Icon(color='blue', icon='info-sign'), popup=f'Cluster {cluster_id} Booth 2').add_to(m)
        folium.PolyLine([booth1, booth2], color='black', weight=2, tooltip=f'{int(dist)} m').add_to(m)

    # Save map with AC name
    filename = f"/Users/amartyaanand/Desktop/assam_as/{ac_name}_map.html"
    m.save(filename)
    print(f"Map saved for AC: {ac_name} at {filename}")


/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:41: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  ac_polygon = ac_boundary.geometry.unary_union
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:49: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = ac_booths_proj.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method i

Map saved for AC: 100 at /Users/amartyaanand/Desktop/assam_as/100_map.html


/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipyker

Map saved for AC: 101 at /Users/amartyaanand/Desktop/assam_as/101_map.html


/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipyker

...

/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipyker

Map saved for AC: 98 at /Users/amartyaanand/Desktop/assam_as/98_map.html
Map saved for AC: 99 at /Users/amartyaanand/Desktop/assam_as/99_map.html


/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:41: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  ac_polygon = ac_boundary.geometry.unary_union
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:49: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = ac_booths_proj.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method i

Map saved for AC: 9 at /Users/amartyaanand/Desktop/assam_as/9_map.html


/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipykernel_24384/2496071029.py:96: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cluster_centroid = cluster_booths.geometry.centroid.unary_union.centroid
/var/folders/fp/dxc22s71695bd7b74zslz2d80000gn/T/ipyker